This Notebook will model the rear irradiance for 200 sensors across a 1 panel system, with torquetube, 

In [1]:
try:
    from bifacial_radiance import *
except ImportError:
    raise RuntimeError('bifacial_radiance is required. download distribution')
    # Simple example system using Radiance.
    
import numpy as np

In [ ]:
# The below routine creates a HEXAGONAL torque tube, for a 2-UP configuration of a specific module size. Parameters for the module, the torque tube, and the scene are below.
# This is being run with gendaylit, for one specific timestamp
# MODIFY THE TEST FOLDER TO YOUR LIKING. I utmostly dislike the interactive directory.
testfolder = r'C:\Users\sayala\Documents\RadianceScenes\Demo4'
writefiletitle = r'C:\Users\sayala\Documents\RadianceScenes\Demo4\results\Fixed_Shading_Compiled_19Mar.csv'
timeindex = 4020 # Noon, June 17th. 
simulationname = 'New_Panel'

import csv

# MakeModule Parameters
moduletype=simulationname
numpanels = 1
x = 1  
y = 2
xgap = 0.01
ygap = 0.0
zgap = 0.02
sensorsy = 200 

# TorqueTube Parameters
axisofrotationTorqueTube=False
torqueTube = True
tubetype = 'square'
diameter = 0.20
torqueTubeMaterial = 'black'       # IT's NOT GRAY, IT's GREY
cellLevelModule = False
numcellsx=0
numcellsy=0

# SceneDict Parameters
pitch = 4
albedo = 0.62 
clearance_height = 0.5  
nMods = 20
nRows = 7
azimuth_ang=180
tilt =30

# Now let's run the example

demo = RadianceObj(simulationname,path = testfolder)  # Create a RadianceObj 'object'
demo.setGround(albedo) # input albedo number or material name like 'concrete'.  To see options, run this without any input.
epwfile = demo.getEPW(37.5,-77.6) # pull TMY data for any global lat/lon
metdata = demo.readEPW(epwfile) # read in the EPW weather data from above

# Making module with all the variables
moduleDict=demo.makeModule(name=moduletype,x=x,y=y,bifi=1, 
           torquetube=torqueTube, diameter = diameter, tubetype = tubetype, 
                           material = torqueTubeMaterial, zgap = zgap, numpanels = numpanels, ygap = ygap, 
                           rewriteModulefile = True, xgap=xgap, 
                           axisofrotationTorqueTube=axisofrotationTorqueTube, cellLevelModule=cellLevelModule, 
                           numcellsx=numcellsx, numcellsy = numcellsy)
# create a scene with all the variables
sceneDict = {'tilt':tilt,'pitch': pitch,'height':clearance_height,'azimuth':azimuth_ang, 'module_type':simulationname, 'nMods': nMods, 'nRows': nRows}  
scene = demo.makeScene(moduletype=moduletype, sceneDict=sceneDict) #makeScene creates a .rad file with 20 modules per row, 7 rows.

with open (writefiletitle,'w') as csvfile:

    sw = csv.writer(csvfile, delimiter=',', quotechar='|', quoting=csv.QUOTE_MINIMAL, lineterminator='\n')

    outputheader=['Latitude(deg)','Longitude(deg)', 'Tilt(deg)']
    outputheadervars=[metdata.latitude, metdata.longitude, sceneDict['tilt']]

    if axisofrotationTorqueTube==False:
        outputheader+=['axisofrotationTorqueTube']
        outputheadervars.append(axisofrotationTorqueTube)

    sw.writerow(outputheader)
    sw.writerow(outputheadervars)
    
    outputheadertitles=['Timestamp','Back Avg', 'FrontAvg', 'Ratio']
    
    xheader=[]
    yheader=[]
    zheader=[]
    zrearheader=[]
    mattypeheader=[]
    rearmattypeheader=[]
    frontIrradheader=[]
    backIrradheader=[]
    import string
    
    for i in range (0, sensorsy):
        xheader.append('x_'+str(i))
        yheader.append('y_'+str(i))
        zheader.append('z_'+str(i))
        zrearheader.append('rearZ_'+str(i))
        mattypeheader.append('mattype_'+str(i))
        rearmattypeheader.append('rearMat_'+str(i))
        frontIrradheader.append('Wm2Front_'+str(i))
        backIrradheader.append('Wm2Back_'+str(i))
    
    outputheadertitles+=xheader
    outputheadertitles+=yheader
    outputheadertitles+=zheader
    outputheadertitles+=zrearheader
    outputheadertitles+=mattypeheader
    outputheadertitles+=rearmattypeheader
    outputheadertitles+=frontIrradheader
    outputheadertitles+=backIrradheader


    sw.writerow(outputheadertitles)

    for timeindex in range (1422,2160):  #2160
        demo.gendaylit(metdata,timeindex)  # Noon, June 17th
        octfile = demo.makeOct(demo.getfilelist())  # makeOct combines all of the ground, sky and object files into a .oct file.
        analysis = AnalysisObj(octfile, demo.name)  # return an analysis object including the scan dimensions for back irradiance

        hubheight= (sceneDict['height']+moduleDict['sceney']/2.0*np.sin(sceneDict['tilt']*np.pi/180))

        frontscan, backscan = analysis.moduleAnalysis(hubheight, sceneDict['azimuth'], sceneDict['tilt'], 
                                                  sceneDict['pitch'], sceneDict['nMods'], sceneDict['nRows'], 
                                                  moduleDict['sceney'], moduleDict['scenex'], moduleDict['moduleoffset'],
                                                  sensorsy=sensorsy)

        frontDict, backDict=analysis.analysis(octfile, "Shading_"+str(timeindex), frontscan, backscan)  # compare the back vs front irradiance
        if frontDict is not None:
            print('Annual bifacial ratio average:  %0.3f \n' %( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) ) )
            outputvalues=[]
            outputvalues.append(timeindex)
            outputvalues.append(sum(analysis.Wm2Back))
            outputvalues.append(sum(analysis.Wm2Front))
            outputvalues.append( sum(analysis.Wm2Back) / sum(analysis.Wm2Front) )
            
            # dictionary
            dictvalues=[]
            dictvalues.append(frontDict['x'])
            dictvalues.append(frontDict['y'])
            dictvalues.append(frontDict['z'])
            dictvalues.append(backDict['z'])
            dictvalues.append(frontDict['mattype'])
            dictvalues.append(backDict['mattype'])
            dictvalues.append(frontDict['Wm2'])
            dictvalues.append(backDict['Wm2'])
            flattened = [val for sublist in dictvalues for val in sublist]
            outputvalues+=flattened
            outputclean = str(outputvalues).translate(string.maketrans('', ''), '[]\'')
            sw.writerow(outputvalues)

print("Finished")


path = C:\Users\sayala\Documents\RadianceScenes\Demo4
Getting weather file: USA_VA_Richmond.Intl.AP.724010_TMY.epw  ... OK!
('\nModule Name:', 'New_Panel')
REWRITING pre-existing module file. 
Module New_Panel successfully created
Adjusting solarposition for sunrise hour, 1422 timeindex
Original datetime 1972-03-01 07:00:00-05:00
Localized and adjusted datetime 1972-03-01 06:50:00-05:00 

Created New_Panel.oct
Linescan in process: Shading_1422_Front
Linescan in process: Shading_1422_Back
Saved: results\irr_Shading_1422.csv
Annual bifacial ratio average:  0.239 

Created New_Panel.oct
Linescan in process: Shading_1423_Front
Linescan in process: Shading_1423_Back
Saved: results\irr_Shading_1423.csv
Annual bifacial ratio average:  0.157 

Created New_Panel.oct
Linescan in process: Shading_1424_Front
Linescan in process: Shading_1424_Back
Saved: results\irr_Shading_1424.csv
Annual bifacial ratio average:  0.135 

Created New_Panel.oct
Linescan in process: Shading_1425_Front
Linescan in pro

Saved: results\irr_Shading_1478.csv
Annual bifacial ratio average:  0.167 

Created New_Panel.oct
Linescan in process: Shading_1479_Front
Linescan in process: Shading_1479_Back
Saved: results\irr_Shading_1479.csv
Annual bifacial ratio average:  0.226 

Created New_Panel.oct
Linescan in process: Shading_1480_Front
Linescan in process: Shading_1480_Back
Saved: results\irr_Shading_1480.csv
Annual bifacial ratio average:  0.237 

Created New_Panel.oct
Linescan in process: Shading_1481_Front
Linescan in process: Shading_1481_Back
Saved: results\irr_Shading_1481.csv
Annual bifacial ratio average:  0.247 

Adjusting solarposition for sunset hour, 1482 timeindex
Original datetime 1972-03-03 19:00:00-05:00
Localized and adjusted datetime 1972-03-03 18:03:00-05:00 

Missing files, skipping...
Analysis aborted - no octfile 

Missing files, skipping...
Analysis aborted - no octfile 

Missing files, skipping...
Analysis aborted - no octfile 

Missing files, skipping...
Analysis aborted - no octfile

Saved: results\irr_Shading_1543.csv
Annual bifacial ratio average:  0.166 

Created New_Panel.oct
Linescan in process: Shading_1544_Front
Linescan in process: Shading_1544_Back
Saved: results\irr_Shading_1544.csv
Annual bifacial ratio average:  0.139 

Created New_Panel.oct
Linescan in process: Shading_1545_Front
Linescan in process: Shading_1545_Back
Saved: results\irr_Shading_1545.csv
Annual bifacial ratio average:  0.124 

Created New_Panel.oct
Linescan in process: Shading_1546_Front
Linescan in process: Shading_1546_Back
Saved: results\irr_Shading_1546.csv
Annual bifacial ratio average:  0.114 

Created New_Panel.oct
Linescan in process: Shading_1547_Front
Linescan in process: Shading_1547_Back
Saved: results\irr_Shading_1547.csv
Annual bifacial ratio average:  0.112 

Created New_Panel.oct
Linescan in process: Shading_1548_Front
Linescan in process: Shading_1548_Back
Saved: results\irr_Shading_1548.csv
Annual bifacial ratio average:  0.108 

Created New_Panel.oct
Linescan in proce

'1422, 738.8577955477602, 3053.5572500000003, 0.24196624954313864, 1.050022e-16, 1.039469e-16, 1.028916e-16, 1.018363e-16, 1.00781e-16, 9.972573e-17, 9.867043e-17, 9.761513e-17, 9.655984e-17, 9.550454e-17, 9.444924e-17, 9.339394e-17, 9.233864e-17, 9.128334e-17, 9.022804e-17, 8.917274e-17, 8.811745e-17, 8.706215e-17, 8.600685e-17, 8.495155e-17, 8.389625e-17, 8.284095e-17, 8.178565e-17, 8.073035e-17, 7.967506e-17, 7.861976e-17, 7.756446e-17, 7.650916e-17, 7.545386e-17, 7.439856e-17, 7.334326e-17, 7.228796e-17, 7.123267e-17, 7.017737e-17, 6.912207e-17, 6.806677e-17, 6.701147e-17, 6.595617e-17, 6.490087e-17, 6.384557e-17, 6.279028e-17, 6.173498e-17, 6.067968e-17, 5.962438e-17, 5.856908e-17, 5.751378e-17, 5.645848e-17, 5.540318e-17, 5.434789e-17, 5.329259e-17, 5.223729e-17, 5.118199e-17, 5.012669e-17, 4.907139e-17, 4.801609e-17, 4.696079e-17, 4.59055e-17, 4.48502e-17, 4.37949e-17, 4.27396e-17, 4.16843e-17, 4.0629e-17, 3.95737e-17, 3.85184e-17, 3.746311e-17, 3.640781e-17, 3.535251e-17, 3.429

[[0.5649751,
  0.5699502,
  0.5749254,
  0.5799005,
  0.5848756,
  0.5898507,
  0.5948259,
  0.599801,
  0.6047761,
  0.6097512,
  0.6147264,
  0.6197015,
  0.6246766,
  0.6296517,
  0.6346269,
  0.639602,
  0.6445771,
  0.6495522,
  0.6545274,
  0.6595025,
  0.6644776,
  0.6694527,
  0.6744279,
  0.679403,
  0.6843781,
  0.6893532,
  0.6943284,
  0.6993035,
  0.7042786,
  0.7092537,
  0.7142289,
  0.719204,
  0.7241791,
  0.7291542,
  0.7341294,
  0.7391045,
  0.7440796,
  0.7490547,
  0.7540299,
  0.759005,
  0.7639801,
  0.7689552,
  0.7739303,
  0.7789055,
  0.7838806,
  0.7888557,
  0.7938308,
  0.798806,
  0.8037811,
  0.8087562,
  0.8137313,
  0.8187065,
  0.8236816,
  0.8286567,
  0.8336318,
  0.838607,
  0.8435821,
  0.8485572,
  0.8535323,
  0.8585075,
  0.8634826,
  0.8684577,
  0.8734328,
  0.878408,
  0.8833831,
  0.8883582,
  0.8933333,
  0.8983085,
  0.9032836,
  0.9082587,
  0.9132338,
  0.918209,
  0.9231841,
  0.9281592,
  0.9331343,
  0.9381095,
  0.9430846,
  0.9480